# Threadsafe

This notebook guides a user on how to enable threadsafe methods when accessing a store in a parallel fashion.


## Lock Strategies

Currently there are three lock strategies available for Kosh that a user can enable.

* `None` (default): Best for local stores (`.sql`, `.sqlite`, etc...) that don't get called in parallel
  * This is the default Lock Strategy which is not using a Lock Strategy at all.
* `kosh.lock_strategies.RFileLock()`: Best for local stores (`.sql`, `.sqlite`, etc...) that get called in parallel every once in a while
  * This locks the store and retries a Kosh method until the store is available
* `kosh.lock_strategies.OnlyRetry()`: Best for cloud stores (`mariadb`) that get called in parallel frequently
  * This only retries Kosh methods since MariaDB has its own lock strategies

A user can enable these different lock strategies by passing them into the `kosh.connect()` parameter `lock_strategy`. A user would then use Kosh like they normally would. That's it!

In [ ]:
import kosh

# Choose lock strategy
# ls = None
ls = kosh.lock_strategies.RFileLock()
# ls = kosh.lock_strategies.OnlyRetry()

store = kosh.connect("threadsafe.sqlite", lock_strategy=ls)

# Commence as usual!
n_ensembles = 10
n_datasets = 10
for i in range(n_ensembles):
    metadata = {f"ensemble_att_{ia}": f"ensemble_{i}_attributes_{ia}" for ia in range(n_ensembles)}
    ens_ID = f"ensemble_{i}"
    ens = store.create_ensemble(id=ens_ID, metadata=metadata)

    for j in range(n_datasets):
        metadata = {f"dataset_att_{ia}": f"dataset_{j}_attributes_{ia}" for ia in range(n_datasets)}
        ds_ID = f"ensemble_{i}_dataset_{j}"
        ds = ens.create(id=ds_ID, metadata=metadata)

## Lock Strategies Parameters

A user can also pass in their own parameters to a lock strategy if they would like.

* `kosh.lock_strategies.RFileLock()`:
  * `num_tries=10`: Number of tries for locking the store file and retrying Kosh method calls
  * `patience=1`: Wait time in seconds between each retry
  * `lock_path=None`: `filelock.FileLock()` file lock path for store (defaults to `os.path.join(os.path.expanduser("~"), ".kosh.lock")`)
  * `timeout=3600`: `filelock.FileLock()` file lock timeout before retrying to lock again
* `kosh.lock_strategies.OnlyRetry()`:
  * `num_tries=10`: Number of tries for locking the store file and retrying Kosh method calls
  * `patience=1`: Wait time in seconds between each retry

In [ ]:
ls = kosh.lock_strategies.RFileLock(num_tries=10, patience=1, lock_path=None, timeout=3600)
store = kosh.connect("threadsafe.sqlite", lock_strategy=ls)